This file aimes to merge several dataframe togethers and create two new dataframes.  
pitch_join2.csv: merged pitch_type information with pitchers pitching statistics  
pitch_join_display.csv: pitch_join2 with selected columns, aims to provide a general table for webpage display

In [1]:
import pandas as pd
import numpy as np
# from pybaseball import statcast
# from pybaseball import pitching_stats_range

pd.set_option('display.max_columns', 100)

In [3]:
# read pre-downloaded data
dat_2017 = pd.read_csv("../input/dat1_2017.csv")
pitch_2017 = pd.read_csv("../input/pitch1_2017.csv")
print(dat_2017.shape, pitch_2017.shape)

(731427, 92) (753, 42)


In [4]:
# Label all pitches that are catergorized as strikeout
dat_2017['strikeout'] = np.where((dat_2017['events'] == 'strikeout') | (dat_2017['events'] == 'strikeout_double_play'), 1,0)
dat_2017['strikeout'].value_counts()

0    690656
1     40771
Name: strikeout, dtype: int64

In [5]:
# group pitches by pitcher, and calculated average speed
col = ['pitch_type', 'release_speed', 'player_name', 'batter', 'pitcher', 'description', 'game_date', 'strikeout']
dat_sel = dat_2017[col]

dat_group = dat_sel.groupby('player_name')
pitch_num = dat_group.size()
speed = dat_group['release_speed'].agg(np.mean)

In [6]:
# Proportion of pitches each pitcher throw
dat_group2 = dat_sel.groupby(['player_name', 'pitch_type'])
group2_per = dat_group2.size().unstack('pitch_type').iloc[:, :-1].apply(lambda x: x / x.sum(), axis = 1)
group2_per['Max'] = group2_per.idxmax(axis = 1)

dat_group3 = dat_sel.groupby(['player_name', 'strikeout'])
group3_per = dat_group3.size().unstack('strikeout').apply(lambda x: x / x.sum(), axis = 1)

group2_per['Pitch_num'] = pitch_num
group2_per['Speed'] = speed
group2_per['strikeout'] = group3_per[1]

In [7]:
group2_per.head(10)

pitch_type,CH,CU,EP,FC,FF,FO,FS,FT,KC,KN,PO,SC,SI,SL,Max,Pitch_num,Speed,strikeout
player_name,,,,,,,,,,,,,,,,,,
A.J. Cole,0.065101,0.177161,NaN,NaN,0.459979,NaN,NaN,0.087513,NaN,NaN,NaN,NaN,NaN,0.210245,FF,937,88.632978,0.046958
A.J. Griffin,0.111279,0.303558,NaN,0.056775,0.517033,NaN,NaN,0.011355,NaN,NaN,NaN,NaN,NaN,NaN,FF,1322,79.773505,0.046142
A.J. Minter,NaN,NaN,NaN,0.487903,0.512097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FF,248,93.095968,0.104839
A.J. Schugel,0.378661,0.052301,NaN,NaN,0.085774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.483264,NaN,SI,496,86.946234,0.054435
AJ Ramos,0.178020,0.023615,NaN,0.008174,0.295186,NaN,NaN,0.064487,NaN,NaN,NaN,NaN,NaN,0.430518,SL,1104,85.439237,0.065217
Aaron Blair,0.120000,0.106667,NaN,NaN,0.066667,NaN,NaN,0.466667,NaN,NaN,NaN,NaN,NaN,0.240000,FT,75,86.737333,0.040000
Aaron Bummer,0.018717,NaN,NaN,NaN,0.163102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.449198,0.368984,SI,374,89.439037,0.045455
Aaron Loup,0.141865,0.102183,NaN,0.102183,0.006944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.646825,NaN,SI,1008,88.031647,0.063492
Aaron Nola,0.155405,0.307808,NaN,NaN,0.324700,NaN,NaN,0.212087,NaN,NaN,NaN,NaN,NaN,NaN,FF,2665,86.222185,0.069043


In [8]:
# important variables for display are selected
pitch_join = pd.merge(pitch_2017.drop(columns ='PO'), group2_per, how = 'inner', right_index = True, left_on = 'Name')
col = ['Name','Age',  'Tm','Max', 'Pitch_num','Speed','strikeout','G', 'GS', 'W', 'L', 'IP', 'Lev',] + list(group2_per)[0:14]
pitch_join_new = pitch_join[col]
pitch_join_new = pitch_join_new[pitch_join['IP'] > 10]
pitch_join_new.head(10)

,Name,Age,Tm,Max,Pitch_num,Speed,strikeout,G,GS,W,L,IP,Lev,CH,CU,EP,FC,FF,FO,FS,FT,KC,KN,PO,SC,SI,SL
0,Fernando Abad,31,Boston,FF,769,84.734070,0.048114,48,0,2.0,1.0,43.2,MLB-AL,0.224967,NaN,NaN,NaN,0.288687,NaN,NaN,0.235371,0.250975,NaN,NaN,NaN,NaN,NaN
2,Tim Adleman,29,Cincinnati,FF,2088,86.412242,0.051724,30,20,5.0,11.0,122.1,MLB-NL,0.199232,0.151704,NaN,NaN,0.551128,NaN,NaN,0.096495,NaN,NaN,0.001440,NaN,NaN,NaN
3,Andrew Albers,31,Seattle,SI,723,83.847165,0.051176,9,6,5.0,1.0,41.0,MLB-AL,0.077455,0.081604,NaN,NaN,0.248963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.398340,0.193638
4,Matt Albers,34,Washington,FF,1007,90.998795,0.062562,63,0,7.0,2.0,61.0,MLB-NL,0.059237,0.001004,NaN,NaN,0.375502,NaN,NaN,NaN,NaN,NaN,0.001004,NaN,0.298193,0.265060
5,Al Alburquerque,31,"Kansas City,Chicago",SL,263,90.382129,0.053232,21,0,NaN,2.0,18.0,MLB-AL,0.019011,NaN,NaN,NaN,0.133080,NaN,NaN,0.304183,NaN,NaN,NaN,NaN,NaN,0.543726
6,Raul Alcantara,24,Oakland,FF,444,92.029571,0.027027,8,4,1.0,2.0,24.0,MLB-AL,0.187359,0.020316,NaN,NaN,0.372460,NaN,NaN,0.248307,NaN,NaN,NaN,NaN,NaN,0.171558
9,Scott Alexander,27,Kansas City,SI,1059,92.767233,0.055713,58,0,5.0,4.0,69.0,MLB-AL,0.017941,NaN,NaN,NaN,0.119924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.819641,0.042493
10,Cody Allen,28,Cleveland,FF,1277,89.772257,0.078309,69,0,3.0,7.0,67.1,MLB-AL,NaN,NaN,NaN,NaN,0.555818,NaN,NaN,NaN,0.444182,NaN,NaN,NaN,NaN,NaN
12,Dan Altavilla,24,Seattle,FF,768,93.437778,0.067708,41,0,1.0,1.0,46.2,MLB-AL,NaN,NaN,NaN,NaN,0.597132,NaN,NaN,0.029987,NaN,NaN,NaN,NaN,NaN,0.372881
13,Jose Alvarado,22,Tampa Bay,FF,463,95.274242,0.062635,35,0,NaN,3.0,29.2,MLB-AL,NaN,0.241901,NaN,NaN,0.598272,NaN,NaN,0.159827,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
pitch_join_new[pitch_join_new['IP'] > 10].to_csv("../output/pitch_join_display.csv", index = False)

In [10]:
pitch_join[pitch_join['IP'] > 10].to_csv("../output/pitch_join2.csv", index = False)